## Installing dependencies

In [1]:
conda install -c conda-forge matplotlib tqdm

Solving environment: | ^C
failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


## Imports

In [2]:
import requests # to make TMDB API calls
import locale
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from tqdm import tqdm

api_key = "102470fa345354ddc5b12075428d47b5"

## Get top 100 movies per each year/last 25 years

In [19]:
current_year = 1996
end_year = 2021

columns = ['id', 'title', 'release_date']
highest_revenue_df= pd.DataFrame(columns=columns)
while current_year <= end_year:
    page_number = 1
    total_pages = 1
    while page_number <= total_pages and page_number <= 5:
        response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' + api_key + '&primary_release_year=' + str(current_year) + '&sort_by=revenue.desc&page=' + str(page_number))
        response_json = response.json();
        total_pages = response_json['total_pages']
        response_mapped = list(map(lambda result: {'id': result['id'], 'title': result['title'], 'release_date': result['release_date']}, response_json['results']))
        df_current = pd.DataFrame(data = response_mapped, columns=columns)
        highest_revenue_df = highest_revenue_df.append(df_current)
        page_number += 1
    current_year+=1
display(highest_revenue_df)


,id,title,release_date
0,602,Independence Day,1996-06-25
1,664,Twister,1996-05-10
2,954,Mission: Impossible,1996-05-22
3,9802,The Rock,1996-06-07
4,10545,The Hunchback of Notre Dame,1996-06-21
...,...,...,...
15,589761,Chernobyl: Abyss,2021-04-15
16,630240,Titane,2021-07-14
17,606236,Family Swap,2021-06-30
18,860878,Qismat 2,2021-09-23


## Getting movie details (highest grossing movies)

In [20]:
columns = ['film', 'revenue', 'release_date']
df = pd.DataFrame(columns=columns)
for index, film in tqdm(highest_revenue_df.iterrows(), total=len(highest_revenue_df), desc='Downloading'):
    film_revenue = requests.get('https://api.themoviedb.org/3/movie/'+ str(film['id']) +'?api_key='+ api_key+'&language=en-US')
    film_revenue = film_revenue.json()
    df.loc[len(df)]=[film['title'],film_revenue['revenue'],film['release_date']]

Downloading: 100%|████████████████████████████| 2600/2600 [06:52<00:00,  6.30it/s]


In [21]:
df

,film,revenue,release_date
0,Independence Day,817400891,1996-06-25
1,Twister,494471524,1996-05-10
2,Mission: Impossible,457731198,1996-05-22
3,The Rock,335062621,1996-06-07
4,The Hunchback of Notre Dame,325338851,1996-06-21
...,...,...,...
2595,Chernobyl: Abyss,5370393,2021-04-15
2596,Titane,5115725,2021-07-14
2597,Family Swap,4849622,2021-06-30
2598,Qismat 2,4700000,2021-09-23


## Export as CSV

In [22]:
df.to_csv('tmdb_api_export_1996_2021_v2.csv')